- network building
- circular visualization
- bar plot node strenght
- save adiacence matrix


In [ ]:
#folium network
import pandas as pd
import numpy as np
import folium
from statsmodels.tsa.stattools import grangercausalitytests
import os
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, to_hex

# Ensure plots directory exists
plots_dir = 'plots'
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Load data
data = pd.read_csv('0.SpeedCT_Hr.csv')

# Load COOR data
coor_data = pd.read_csv('CoorTraffic.csv')

# Extract the start time from the 'Time' range
data['StartTime'] = data['Time'].str.split('-').str[0]

# Combine 'Date' and 'StartTime' to create 'DateTime'
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['StartTime'], format='%d.%m.%y %H:%M')

# Drop the original 'Date' and 'Time' columns
data.drop(columns=['Date', 'Time', 'StartTime'], inplace=True)

# Set the maximum lag
max_lag = 1  # Change lag

# Filter for Tuesdays, Wednesdays, and Thursdays
data['DayOfWeek'] = data['DateTime'].dt.dayofweek
filtered_data = data[data['DayOfWeek'].isin([6])]  # 0=Monday, 1=Tuesday, 2=Wednesday, 3=Thursday, 4=Friday

# Filter for specific hours (e.g., 6 AM to 10 AM)
#filtered_data['Hour'] = filtered_data['DateTime'].dt.hour
#filtered_data = filtered_data[(filtered_data['Hour'] >= 6) & (filtered_data['Hour'] <= 10)]

# Handle duplicates by averaging the speeds for the same DateTime and ID
filtered_data = filtered_data.groupby(['DateTime', 'ID']).agg({'H.Avg Speed': 'mean'}).reset_index()

# Granger causality test function
def granger_test(dataframe, column1, column2, max_lag):
    try:
        dataframe = dataframe.dropna(subset=[column1, column2])
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[max_lag][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[max_lag][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.01 else 0
    except Exception as e:
        print(f"Error in granger_test: {e}")
        return 0

# Function to create a Folium network map
def create_folium_network_map(title, filename, node_positions, outlink_strength_total, inlink_strength_total):
    # Initialize the map centered around the mean coordinates
    map_center = [coor_data['NORD'].mean(), coor_data['EST'].mean()]
    m = folium.Map(location=map_center, zoom_start=13)

    # Normalize outlink and inlink strengths using the total strengths
    outlink_min, outlink_max = min(outlink_strength_total.values()), max(outlink_strength_total.values())
    inlink_min, inlink_max = min(inlink_strength_total.values()), max(inlink_strength_total.values())

    outlink_strength_normalized = {station: (outlink_strength_total[station] - outlink_min) / (outlink_max - outlink_min) if outlink_max != outlink_min else 0 for station in outlink_strength_total}
    inlink_strength_normalized = {station: (inlink_strength_total[station] - inlink_min) / (inlink_max - inlink_min) if inlink_max != inlink_min else 0 for station in inlink_strength_total}

    # Setup color normalization and colormap
    norm = Normalize(vmin=0, vmax=1)
    cmap = plt.cm.viridis

    # Add nodes to the map
    for station, (x, y) in node_positions.items():
        if station in outlink_strength_normalized and station in inlink_strength_normalized:
            size = 2 + outlink_strength_normalized[station] * 6  # Node size based on outlink strength
            color = to_hex(cmap(norm(inlink_strength_normalized[station])))  # Node color based on inlink strength
            marker = folium.CircleMarker(location=[y, x], radius=size, color=color, fill=True, fill_color=color, fill_opacity=0.6, popup=station) if coor_data[coor_data['IDDI'] == station]['Direction'].values[0] == 'IN' else folium.RegularPolygonMarker(location=[y, x], radius=size, color=color, fill=True, fill_color=color, fill_opacity=0.6, popup=station, number_of_sides=5)
            marker.add_to(m)

    # Save the map
    m.save(filename)
    print(f"Saved map to {filename}")

# Create node positions dictionary for plotting using the full coor_data
node_positions = {row['IDDI']: (row['EST'], row['NORD']) for _, row in coor_data.iterrows()}

# Initialize total strengths
outlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}
inlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}

# Pivot the filtered data
pivot_data = filtered_data.pivot(index='DateTime', columns='ID', values='H.Avg Speed').fillna(0)

# Analysis for each station pair
for station1 in pivot_data.columns:
    for station2 in pivot_data.columns:
        if station1 != station2:
            f_statistic_out = granger_test(pivot_data, station1, station2, max_lag)
            if f_statistic_out > 0:
                outlink_strength_total[station1] += f_statistic_out
                inlink_strength_total[station2] += f_statistic_out

# Create network map
title = 'Granger Causality Network'
filename = f'{plots_dir}/3.AvgSpeedHr_Sun.html'
create_folium_network_map(title, filename, node_positions, outlink_strength_total, inlink_strength_total)

# Print total strengths
print("Total Outlink Strength:", outlink_strength_total)
print("Total Inlink Strength:", inlink_strength_total)

In [ ]:
# circlular network
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, to_hex
from statsmodels.tsa.stattools import grangercausalitytests
import os

# Ensure plots directory exists
plots_dir = 'plots'
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Load data
data = pd.read_csv('0.TrafficCT.csv')

# Load COOR data
coor_data = pd.read_csv('CoorTraffic.csv')

# Extract the start time from the 'Time' range
data['StartTime'] = data['Time'].str.split('-').str[0]

# Combine 'Date' and 'StartTime' to create 'DateTime'
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['StartTime'], format='%d.%m.%y %H:%M')

# Drop the original 'Date' and 'Time' columns
data.drop(columns=['Date', 'Time', 'StartTime'], inplace=True)

# Set the maximum lag
max_lag = 1  # Change lag

# Filter for specific day (e.g., Monday)
data['DayOfWeek'] = data['DateTime'].dt.dayofweek
filtered_data = data[data['DayOfWeek'].isin([6])]  # 0=Monday

# Handle duplicates by averaging the speeds for the same DateTime and ID
filtered_data = filtered_data.groupby(['DateTime', 'ID']).agg({'Congestion Factor': 'mean'}).reset_index()

# Granger causality test function
def granger_test(dataframe, column1, column2, max_lag):
    try:
        dataframe = dataframe.dropna(subset=[column1, column2])
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[max_lag][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[max_lag][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.01 else 0
    except Exception as e:
        print(f"Error in granger_test: {e}")
        return 0

# Initialize total strengths
outlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}
inlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}

# Pivot the filtered data
pivot_data = filtered_data.pivot(index='DateTime', columns='ID', values='Congestion Factor').fillna(0)

# Create a graph
G = nx.DiGraph()

# Collect all F statistics
f_statistics = []

# Analysis for each station pair and add edges to the graph
edges = []
for station1 in pivot_data.columns:
    for station2 in pivot_data.columns:
        if station1 != station2:
            f_statistic_out = granger_test(pivot_data, station1, station2, max_lag)
            if f_statistic_out > 0:
                outlink_strength_total[station1] += f_statistic_out
                inlink_strength_total[station2] += f_statistic_out
                edges.append((station1, station2, f_statistic_out))
                f_statistics.append(f_statistic_out)

# Filter edges for the top ToT% F-statistics
threshold = np.percentile([f for _, _, f in edges], 95)
filtered_edges = [(u, v, f) for u, v, f in edges if f >= threshold]

# Add nodes and filtered edges to the graph
G.add_nodes_from(pivot_data.columns)
G.add_weighted_edges_from(filtered_edges)

# Create a circular layout
pos = nx.circular_layout(G)

# Normalize edge weights for color mapping
norm = Normalize(vmin=min(f for _, _, f in filtered_edges), vmax=max(f for _, _, f in filtered_edges))

cmap = plt.cm.viridis

# Draw the graph
plt.figure(figsize=(12, 12))
nx.draw_networkx_nodes(G, pos, node_size=100, node_color='lightblue')
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')
nx.draw_networkx_edges(G, pos, edgelist=filtered_edges, edge_color=[to_hex(cmap(norm(f))) for _, _, f in filtered_edges], arrowstyle='-|>', arrowsize=15, width=0.6)

# Create colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.set_label('F-statistic')

# Save the plot
#plt.title('Granger Causality Network for Mondays')
plt.savefig(f'{plots_dir}/3.Circular_CongestionFactor_Sun.jpg', dpi=300)
plt.show()

# Plot the histogram of F statistics
plt.figure(figsize=(10, 6))
plt.hist(f_statistics, bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram of F Statistics - Congestion Factor')
plt.xlabel('F Statistic')
plt.ylabel('Frequency')
plt.grid(True)

# Save the histogram as a JPG image with 300 dpi
plt.savefig(f'{plots_dir}/3.f_statistics_histogramCongestionFactor_Sun.jpg', format='jpg', dpi=300)
plt.show()


In [ ]:
# bar plot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
from matplotlib.lines import Line2D
import os

# Ensure plots directory exists
if not os.path.exists('plots'):
    os.makedirs('plots')

# Load data
data = pd.read_csv('0.SpeedCT_Hr.csv')

# Load COOR data
coor_data = pd.read_csv('CoorTraffic.csv')

# Extract the start time from the 'Time' range
data['StartTime'] = data['Time'].str.split('-').str[0]

# Combine 'Date' and 'StartTime' to create 'DateTime'
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['StartTime'], format='%d.%m.%y %H:%M')

# Drop the original 'Date', 'Time', and 'StartTime' columns
data.drop(columns=['Date', 'Time', 'StartTime'], inplace=True)

# Filter for specific days (e.g., Monday to Friday)
data['DayOfWeek'] = data['DateTime'].dt.dayofweek
filtered_data = data[data['DayOfWeek'].isin([6])]  # 0=Monday, 1=Tuesday, 2=Wednesday, 3=Thursday, 4=Friday

# Handle duplicates by averaging the speeds for the same DateTime and ID
filtered_data = filtered_data.groupby(['DateTime', 'ID']).agg({'H.Avg Speed': 'mean'}).reset_index()

# Set the maximum lag
max_lag = 1  # Change lag

# Granger causality test function
def granger_test(dataframe, column1, column2, max_lag):
    try:
        dataframe = dataframe.dropna(subset=[column1, column2])
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[max_lag][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[max_lag][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.01 else 0
    except Exception as e:
        print(f"Error in granger_test: {e}")
        return 0

# Loop over the entire dataset for analysis
outlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}
inlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}

# Pivot the filtered data
pivot_data = filtered_data.pivot(index='DateTime', columns='ID', values='H.Avg Speed').fillna(0)

links = {}
outlink_strength = {station: 0 for station in filtered_data['ID'].unique()}
inlink_strength = {station: 0 for station in filtered_data['ID'].unique()}

# Analysis for each station pair
for station1 in pivot_data.columns:
    for station2 in pivot_data.columns:
        if station1 != station2:
            f_statistic_out = granger_test(pivot_data, station1, station2, max_lag)
            if f_statistic_out > 0:
                links[(station1, station2)] = f_statistic_out
                outlink_strength[station1] += f_statistic_out
                inlink_strength[station2] += f_statistic_out

# Update total strengths
for station in filtered_data['ID'].unique():
    outlink_strength_total[station] += outlink_strength[station]
    inlink_strength_total[station] += inlink_strength[station]

# Merge COOR data with data to get coordinates
merged_data = pd.merge(filtered_data, coor_data, left_on='ID', right_on='IDDI', how='left')

# Function to plot total strengths and save the plot with specified colors for IN and OUT nodes
def plot_total_strengths(total_strength, title, ylabel, filename, dpi=300):
    sorted_nodes = sorted(total_strength.items(), key=lambda x: x[1], reverse=True)
    node_ids = [node[0] for node in sorted_nodes]
    node_strengths = [node[1] for node in sorted_nodes]

    # Ensure all node IDs have names
    node_names = []
    for node in node_ids:
        name_row = merged_data[merged_data['ID'] == node]
        if not name_row.empty:
            node_names.append(name_row['Name'].iloc[0])
        else:
            node_names.append(f'Node {node}')

    # Plot total strengths
    plt.figure(figsize=(12, 6))
    bars = plt.bar(np.arange(len(node_names)), node_strengths, color=['red' if merged_data[merged_data['ID'] == node]['Direction'].iloc[0] == 'OUT' else 'blue' for node in node_ids])

    plt.ylabel(ylabel)
    plt.title(title)
    plt.xticks(np.arange(len(node_names)), node_names, rotation=90, ha='right', fontsize=4)
    plt.tight_layout()

    # Create legend for IN and OUT nodes
    legend_labels = {'OUT': 'red', 'IN': 'blue'}

    plt.legend(handles=[Line2D([0, 1], [0, 0], color=color, linewidth=3) for color in legend_labels.values()],
               labels=list(legend_labels.keys()), loc='upper left', bbox_to_anchor=(1, 1))

    # Save plot with specified DPI
    plt.savefig(filename, dpi=dpi, bbox_inches='tight')
    plt.close()

# Plot total outlink strengths for the entire dataset and save the plot
outlink_filename = 'plots/3.total_outlink_strengthAvgSpeedHr_Sun.jpg'
plot_total_strengths(outlink_strength_total, 'Total Outlink Strength per Node - H.Avg Speed Hr', 'Total Outlink Strength', outlink_filename)

# Plot total inlink strengths for the entire dataset and save the plot
inlink_filename = 'plots/3.total_intlink_strengtAvgSpeedHr_Sun.jpg'
plot_total_strengths(inlink_strength_total, 'Total Inlink Strength per Node - H.Avg Speed Hr', 'Total Inlink Strength', inlink_filename)

# Print total strengths
print("Total Outlink Strength:", outlink_strength_total)
print("Total Inlink Strength:", inlink_strength_total)


In [ ]:
#bar plot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
import os

# Ensure plots directory exists
if not os.path.exists('plots'):
    os.makedirs('plots')

# Load data
data = pd.read_csv('0.SpeedCT_15m.csv')

# Load COOR data
coor_data = pd.read_csv('CoorTraffic.csv')

# Extract the start time from the 'Time' range
data['StartTime'] = data['Time'].str.split('-').str[0]

# Combine 'Date' and 'StartTime' to create 'DateTime'
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['StartTime'], format='%d.%m.%y %H:%M')

# Drop the original 'Date', 'Time', and 'StartTime' columns
data.drop(columns=['Date', 'Time', 'StartTime'], inplace=True)

# Filter for specific days (e.g., Tuesday, Wednesday, and Thursday)
data['DayOfWeek'] = data['DateTime'].dt.dayofweek
filtered_data = data[data['DayOfWeek'].isin([0,1,2,3,4])]  # 0=Monday, 1=Tuesday, 2=Wednesday, 3=Thursday


# Filter for specific hours (e.g., 6 AM to 10 AM)
#filtered_data['Hour'] = filtered_data['DateTime'].dt.hour
#filtered_data = filtered_data[(filtered_data['Hour'] >= 6) & (filtered_data['Hour'] <= 10)]


# Handle duplicates by averaging the speeds for the same DateTime and ID
filtered_data = filtered_data.groupby(['DateTime', 'ID']).agg({'H.Avg Speed': 'mean'}).reset_index()



# Set the maximum lag
max_lag = 1  # This means we are considering 1 hour lag (4 * 15 minutes)

# Granger causality test function
def granger_test(dataframe, column1, column2, max_lag):
    try:
        dataframe = dataframe.dropna(subset=[column1, column2])
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[max_lag][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[max_lag][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.01 else 0
    except Exception as e:
        print(f"Error in granger_test: {e}")
        return 0

# Loop over the entire dataset for analysis
outlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}
inlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}

# Color assignment based on sorted node IDs
sorted_nodes = sorted(filtered_data['ID'].unique())
color_palette = plt.cm.get_cmap('tab20', len(sorted_nodes))
node_colors = {node: color_palette(i) for i, node in enumerate(sorted_nodes)}

# Function to plot total strengths and save the plot
def plot_total_strengths(total_strength, node_colors, title, ylabel, filename, dpi=300):
    sorted_nodes = sorted(total_strength.items(), key=lambda x: x[1], reverse=True)
    node_ids = [node[0] for node in sorted_nodes]
    node_strengths = [node[1] for node in sorted_nodes]

    # Ensure all node IDs have names
    node_names = []
    for node in node_ids:
        name_row = merged_data[merged_data['ID'] == node]
        if not name_row.empty:
            node_names.append(name_row['Name'].iloc[0])
        else:
            node_names.append(f'Node {node}')

    # Plot total strengths
    plt.figure(figsize=(12, 6))
    bars = plt.bar(np.arange(len(node_names)), node_strengths, color=[node_colors[node] for node in node_ids])

    # Add hatch to "IN" nodes
    for bar, node in zip(bars, node_ids):
        direction = merged_data[merged_data['ID'] == node]['Direction'].iloc[0]
        if direction == 'IN':
            bar.set_hatch('*')

    plt.ylabel(ylabel)
    plt.title(title)
    plt.xticks(np.arange(len(node_names)), node_names, rotation=90, ha='right', fontsize=4)
    plt.tight_layout()

    # Create legend based on IDDI part of node names
    legend_labels = {}
    for node_id, color in node_colors.items():
        node_name = merged_data[merged_data['ID'] == node_id]['Name'].iloc[0] if not merged_data[merged_data['ID'] == node_id].empty else f'Node {node_id}'
        if isinstance(node_name, str):
            iddi_part = node_name.split(',')[0]  # Extract IDDI part of the name
        else:
            iddi_part = f'Node {node_id}'
        if iddi_part not in legend_labels:
            legend_labels[iddi_part] = color

    plt.legend(handles=[Line2D([0, 1], [0, 0], color=color, linewidth=3) for color in legend_labels.values()],
               labels=list(legend_labels.keys()), loc='upper left', bbox_to_anchor=(1, 1))

    # Save plot with specified DPI
    plt.savefig(filename, dpi=dpi, bbox_inches='tight')
    plt.close()

# Pivot the filtered data
pivot_data = filtered_data.pivot(index='DateTime', columns='ID', values='H.Avg Speed').fillna(0)

links = {}
outlink_strength = {station: 0 for station in filtered_data['ID'].unique()}
inlink_strength = {station: 0 for station in filtered_data['ID'].unique()}

# Analysis for each station pair
for station1 in pivot_data.columns:
    for station2 in pivot_data.columns:
        if station1 != station2:
            f_statistic_out = granger_test(pivot_data, station1, station2, max_lag)
            if f_statistic_out > 0:
                links[(station1, station2)] = f_statistic_out
                outlink_strength[station1] += f_statistic_out
                inlink_strength[station2] += f_statistic_out

# Update total strengths
for station in filtered_data['ID'].unique():
    outlink_strength_total[station] += outlink_strength[station]
    inlink_strength_total[station] += inlink_strength[station]

# Merge COOR data with data to get coordinates
merged_data = pd.merge(filtered_data, coor_data, left_on='ID', right_on='IDDI', how='left')

# Plot total outlink strengths for the entire dataset and save the plot
outlink_filename = 'plots/5.total_outlink_strength_MontoFri.jpg'
plot_total_strengths(outlink_strength_total, node_colors, 'Total Outlink Strength per Node', 'Total Outlink Strength', outlink_filename)

# Plot total inlink strengths for the entire dataset and save the plot
inlink_filename = 'plots/5.total_inlink_strength_MontoFri.jpg'
plot_total_strengths(inlink_strength_total, node_colors, 'Total Inlink Strength per Node', 'Total Inlink Strength', inlink_filename)

# Print total strengths
print("Total Outlink Strength:", outlink_strength_total)
print("Total Inlink Strength:", inlink_strength_total)


In [ ]:
# adiancence matrix
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
import os

# Ensure plots directory exists
plots_dir = 'plots'
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Load data
data = pd.read_csv('0.SpeedCT_15m.csv')

# Load COOR data
coor_data = pd.read_csv('CoorTraffic.csv')

# Extract the start time from the 'Time' range
data['StartTime'] = data['Time'].str.split('-').str[0]

# Combine 'Date' and 'StartTime' to create 'DateTime'
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['StartTime'], format='%d.%m.%y %H:%M')

# Drop the original 'Date' and 'Time' columns
data.drop(columns=['Date', 'Time', 'StartTime'], inplace=True)

# Set the maximum lag
max_lag = 1  # This means we are considering 1 hour lag (4 * 15 minutes)

# Filter for Tuesdays, Wednesdays, and Thursdays
data['DayOfWeek'] = data['DateTime'].dt.dayofweek
filtered_data = data[data['DayOfWeek'].isin([5])]  # 0=Monday, 1=Tuesday, 2=Wednesday, 3=Thursday, 4=Friday

# Handle duplicates by averaging the speeds for the same DateTime and ID
filtered_data = filtered_data.groupby(['DateTime', 'ID']).agg({'H.Avg Speed': 'mean'}).reset_index()

# Granger causality test function
def granger_test(dataframe, column1, column2, max_lag):
    try:
        dataframe = dataframe.dropna(subset=[column1, column2])
        gc_test = grangercausalitytests(dataframe[[column1, column2]], maxlag=max_lag)
        f_statistic = gc_test[max_lag][0]['ssr_ftest'][0]  # F-statistic
        p_value = gc_test[max_lag][0]['ssr_ftest'][1]  # p-value
        return f_statistic if p_value < 0.01 else 0
    except Exception as e:
        print(f"Error in granger_test: {e}")
        return 0

# Create node positions dictionary for plotting using the full coor_data
node_positions = {row['IDDI']: (row['EST'], row['NORD']) for _, row in coor_data.iterrows()}

# Initialize total strengths
outlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}
inlink_strength_total = {station: 0 for station in filtered_data['ID'].unique()}

# Pivot the filtered data
pivot_data = filtered_data.pivot(index='DateTime', columns='ID', values='H.Avg Speed').fillna(0)

# Create an adjacency matrix DataFrame
adj_matrix = pd.DataFrame(0, index=pivot_data.columns, columns=pivot_data.columns)

# Analysis for each station pair
for station1 in pivot_data.columns:
    for station2 in pivot_data.columns:
        if station1 != station2:
            f_statistic_out = granger_test(pivot_data, station1, station2, max_lag)
            adj_matrix.loc[station1, station2] = f_statistic_out

# Save the adjacency matrix to a CSV file
adj_matrix_csv_path = f'{plots_dir}/adjacency_matrix_Sat.csv'
adj_matrix.to_csv(adj_matrix_csv_path)
print(f"Saved adjacency matrix to {adj_matrix_csv_path}")

# Print total strengths
print("Total Outlink Strength:", outlink_strength_total)
print("Total Inlink Strength:", inlink_strength_total)
